# Move data in a GCP bucket

In [1]:
import os                                  # os handles directory/workspace changes
from subprocess import Popen               # Run shell commands
import yaml                                # Save and load YAML files
from tqdm.auto import tqdm                 # Progress bar

In [3]:
bucket_name = input('Bucket name:')

Bucket name: preprocessed-thesis-data


In [4]:
# Change to parent directory (presumably "Documents")
os.chdir("../../../..")

In [5]:
# Path to the dataset files
data_path = 'Datasets/Thesis/eICU/cleaned/'

In [6]:
stream_tvt_sets = open(f'{data_path}eICU_tvt_sets.yml', 'r')
eICU_tvt_sets = yaml.load(stream_tvt_sets, Loader=yaml.FullLoader)
eICU_tvt_sets

{'test_indeces': [6565,
  2229,
  7333,
  4471,
  7793,
  1962,
  996,
  5780,
  5026,
  1201,
  773,
  3888,
  5517,
  5284,
  6088,
  1202,
  1652,
  5911,
  4138,
  2003,
  7494,
  4431,
  2328,
  1381,
  290,
  3769,
  3419,
  2530,
  4376,
  3415,
  349,
  6813,
  757,
  3032,
  6760,
  7974,
  7278,
  4086,
  2832,
  7300,
  1638,
  4497,
  1142,
  6744,
  5081,
  1391,
  7979,
  358,
  2607,
  5973,
  1517,
  694,
  812,
  5986,
  1307,
  6778,
  1239,
  7967,
  2801,
  2441,
  7326,
  1150,
  5775,
  2139,
  272,
  1955,
  4515,
  6137,
  1862,
  5282,
  7956,
  2567,
  4453,
  6405,
  2873,
  6853,
  3709,
  2245,
  5627,
  564,
  863,
  6737,
  6528,
  1863,
  6923,
  72,
  2221,
  3747,
  1898,
  5862,
  1213,
  3761,
  5838,
  6682,
  4297,
  4470,
  708,
  2761,
  1602,
  5984,
  682,
  6269,
  3139,
  1850,
  6898,
  7728,
  3652,
  6745,
  6045,
  914,
  2032,
  838,
  5741,
  7852,
  1330,
  4073,
  5513,
  473,
  6530,
  4259,
  1551,
  5315,
  64,
  4025,
  110,
  467

In [7]:
all_subset_files = [val for set_list in eICU_tvt_sets.values() for val in set_list]
all_subset_files

[6565,
 2229,
 7333,
 4471,
 7793,
 1962,
 996,
 5780,
 5026,
 1201,
 773,
 3888,
 5517,
 5284,
 6088,
 1202,
 1652,
 5911,
 4138,
 2003,
 7494,
 4431,
 2328,
 1381,
 290,
 3769,
 3419,
 2530,
 4376,
 3415,
 349,
 6813,
 757,
 3032,
 6760,
 7974,
 7278,
 4086,
 2832,
 7300,
 1638,
 4497,
 1142,
 6744,
 5081,
 1391,
 7979,
 358,
 2607,
 5973,
 1517,
 694,
 812,
 5986,
 1307,
 6778,
 1239,
 7967,
 2801,
 2441,
 7326,
 1150,
 5775,
 2139,
 272,
 1955,
 4515,
 6137,
 1862,
 5282,
 7956,
 2567,
 4453,
 6405,
 2873,
 6853,
 3709,
 2245,
 5627,
 564,
 863,
 6737,
 6528,
 1863,
 6923,
 72,
 2221,
 3747,
 1898,
 5862,
 1213,
 3761,
 5838,
 6682,
 4297,
 4470,
 708,
 2761,
 1602,
 5984,
 682,
 6269,
 3139,
 1850,
 6898,
 7728,
 3652,
 6745,
 6045,
 914,
 2032,
 838,
 5741,
 7852,
 1330,
 4073,
 5513,
 473,
 6530,
 4259,
 1551,
 5315,
 64,
 4025,
 110,
 4678,
 52,
 1878,
 4778,
 42,
 6317,
 4971,
 7002,
 2362,
 4791,
 2588,
 7262,
 5429,
 6433,
 3370,
 6172,
 936,
 5558,
 777,
 5158,
 5006,
 4793

In [8]:
for file_num in tqdm(all_subset_files):
    Popen(f'gsutil mv gs://{bucket_name}/eICU_{file_num}.ftr gs://{bucket_name}/subset', shell=True).wait()